In [ ]:
import pyvista as pv

from kale import Engine, show_ui
from kale.algorithms import contour_banded, subdivide_algorithm

In [ ]:
# Requires https://github.com/pyvista/pyvista/pull/3385
pv.set_jupyter_backend("server")
pv.global_theme.trame.interactive_ratio = 2
pv.global_theme.trame.still_ratio = 2

In [ ]:
MESH_GEOMETRY_FILE_NAME = "2022_11_30_11_52_49_mesh_geometry.vtk"
MESH_VALUES_FILE_NAME = "2022_11_30_11_52_49.hdf"
MESH_Z_SCALE = 0.01
TIME_STEP_INTERESTING = 55849
N_CONTOURS = 10
LINE_WIDTH = 1
COLOR_MAP = "CET_L19"
CONTOUR_LINE_COLOR = "black"

## Load Data

Set up the driver of the data IO and time step management with the `kale.Engine` class

In [ ]:
engine = Engine(MESH_GEOMETRY_FILE_NAME, MESH_VALUES_FILE_NAME, zscale=MESH_Z_SCALE)
engine.keys

In [ ]:
# Set which array to be active
engine.mesh.active_scalars_name = "cumulative_slip"
engine.time_step = TIME_STEP_INTERESTING

## Plot with time controls

In [ ]:
pl = pv.Plotter()
pl.add_mesh(
    engine.algorithm,
    cmap=COLOR_MAP,
    clim=engine.clim(),
)
pl.add_mesh(engine.boundary, color=CONTOUR_LINE_COLOR, line_width=LINE_WIDTH)
show_ui(engine, pl)

## Banded Contouring

In [ ]:
contour, edges = contour_banded(
    # engine.algorithm,
    engine.algorithm_smoothed,  # we can play with the subdividing parameters if needed
    N_CONTOURS,
    rng=engine.clim(),
    scalars=engine.mesh.active_scalars_name,
)

In [ ]:
pl = pv.Plotter()
pl.add_mesh(engine.boundary_smoothed, color="k", line_width=LINE_WIDTH)
pl.add_mesh(
    contour,
    cmap=COLOR_MAP,
    clim=engine.clim(),
    scalars="cumulative_slip",
    n_colors=N_CONTOURS - 1,
)
pl.add_mesh(edges.GetOutputPort(1), color=CONTOUR_LINE_COLOR, line_width=LINE_WIDTH)
show_ui(engine, pl)

### 4-panel view (all variables)

This still needs some optimization to make look really nice but here is a start

In [ ]:
engine.keys

In [ ]:
pl = pv.Plotter(shape=(1, 4), window_size=(1024 * 2, 768))

for i, var in enumerate(engine.keys):
    pl.subplot(0, i)
    contour, edges = contour_banded(
        engine.algorithm_smoothed, N_CONTOURS, rng=engine.clim(), scalars=var
    )
    pl.add_mesh(contour, cmap="CET_C3", clim=engine.clim(var), n_colors=N_CONTOURS - 1)
    pl.add_mesh(edges.GetOutputPort(1), color=CONTOUR_LINE_COLOR, line_width=LINE_WIDTH)
    pl.add_mesh(engine.boundary, color=CONTOUR_LINE_COLOR, line_width=LINE_WIDTH)

pl.link_views()
show_ui(engine, pl)